In [1]:
IMAGE_DIMS = (50,34,50,1)
OUTPUT_DIR = '../../output/train-local/'
INPUT_DIR = '../../input/stage1_prepared_some/'
LOAD_MODEL_FILE = OUTPUT_DIR + 'tf-checkpoint-best7826'

In [2]:
import tflearn
from tflearn.layers.core import *
from tflearn.layers.conv import *
from tflearn.data_utils import *
from tflearn.layers.normalization import *
from tflearn.layers.estimator import regression
import os
import shutil
import datetime
import h5py
import logging

In [3]:
logger = logging.getLogger()
formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
sh = logging.StreamHandler()
sh.setLevel(logging.INFO)
sh.setFormatter(formatter)
logger.addHandler(sh)

def setup_file_logger(log_file):
    hdlr = logging.FileHandler(log_file)
    hdlr.setLevel(logging.DEBUG)
    hdlr.setFormatter(formatter)
    logger.addHandler(hdlr) 
    logger.setLevel(logging.DEBUG)

In [4]:
def load_data(input_dir, name, image_dims):
    return h5py.File('{}{}-centered-rotated-{}-{}-{}.h5'.format(input_dir,name,image_dims[2],image_dims[1],image_dims[0]), 'r')

In [5]:
def prepare_dirs(output_dir):
    logger.info('Preparing output dir')

    dir_tflogs = output_dir + 'tf-logs'
    dir_checkpoints = output_dir + 'tf-checkpoint'
    dir_checkpoint_best = output_dir + 'tf-checkpoint-best'
    
#     shutil.rmtree(output_dir, True)
    try:
        os.makedirs(dir_tflogs)
    except:
        pass

    return dir_tflogs, dir_checkpoints, dir_checkpoint_best

In [6]:
def evaluate_test_dataset(input_dir, image_dims):
    with load_data(input_dir, 'test', image_dims) as hdf5:
        X = hdf5['X']
        Y = hdf5['Y']
        logger.debug('X_test shape ' + str(X.shape))
        logger.debug('Y_test shape ' + str(Y.shape))
            
        logger.info('Evaluate performance on test set...')
        acc = model.evaluate(X, Y, batch_size=12)
        logger.info('Score on test dataset: ' + str(acc))

In [7]:
def network(image_dims):
    net = input_data(shape=[None, image_dims[0], image_dims[1], image_dims[2], image_dims[3]], dtype=tf.float32)
    
    net = conv_3d(net, 32, 3, strides=1, activation='relu')
    net = max_pool_3d(net, [1,2,2,2,1], strides=[1,2,2,2,1])

    net = conv_3d(net, 64, 3, strides=1, activation='relu')
    net = max_pool_3d(net, [1,2,2,2,1], strides=[1,2,2,2,1])
    
    net = fully_connected(net, 64, activation='relu')
    net = dropout(net, 0.8)
    
    net = fully_connected(net, 2, activation='softmax')
    
    net = regression(net, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.001)
    return net

In [8]:
def prepare_cnn(output_dir, image_dims):
    dir_tflogs, dir_checkpoints, dir_checkpoint_best = prepare_dirs(output_dir)
    setup_file_logger(output_dir + 'out.log')

    logger.info('Preparing CNN network...')
    net = network(image_dims)

    logger.info('Preparing DNN trainer...')
    model = tflearn.DNN(net, tensorboard_verbose=3, 
                             tensorboard_dir=dir_tflogs,
                             checkpoint_path=dir_checkpoints,
                             best_checkpoint_path=dir_checkpoint_best)
    return model

In [9]:
def start_training(model, input_dir, image_dims):

    with load_data(input_dir, 'train', image_dims) as train_hdf5:
        X = train_hdf5['X']
        Y = train_hdf5['Y']
        logger.info('X shape ' + str(X.shape))
        logger.info('Y shape ' + str(Y.shape))

        with load_data(input_dir, 'validate', image_dims) as validate_hdf5:
            X_validate = validate_hdf5['X']
            Y_validate = validate_hdf5['Y']
            logger.info('X_validate shape ' + str(X_validate.shape))
            logger.info('Y_validate shape ' + str(Y_validate.shape))

            logger.info('Starting CNN training...')
            model.fit(X, Y, validation_set=(X_validate, Y_validate), 
                      shuffle=True, batch_size=10, n_epoch=2,
                      show_metric=True, 
                      run_id='first'+str(image_dims))

    evaluate_test_dataset(input_dir, image_dims)


In [10]:
logger.info('PREPARE CNN')
model = prepare_cnn(OUTPUT_DIR, IMAGE_DIMS)

2017-02-20 04:05:14,703 INFO Preparing CNN network...
2017-02-20 04:05:14,821 INFO Preparing DNN trainer...


In [11]:
if(LOAD_MODEL_FILE is not None):
    logger.info('LOAD PREVIOUS TRAINING')
    model.load(LOAD_MODEL_FILE)
    evaluate_test_dataset(INPUT_DIR, IMAGE_DIMS)

2017-02-20 04:05:16,597 INFO LOAD PREVIOUS TRAINING
2017-02-20 04:05:17,978 INFO Evaluate performance on test set...
2017-02-20 04:05:28,587 INFO Score on test dataset: [0.82608694356420764]


In [12]:
# logger.info('TRAIN CNN')
# start_training(model, INPUT_DIR, IMAGE_DIMS)
# model.save(OUTPUT_DIR + 'final')
# logger.info('==== ALL DONE ====')